# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print("Current working directory is: " + os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
# reference for "glob.glob": https://docs.python.org/3/library/glob.html
    file_path_list = glob.glob(os.path.join(root,'*'))
print("Total number of files is: ", len(file_path_list))

Current working directory is: /home/workspace
Total number of files is:  30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print("Total number of rows is: ", len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that 
# will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length', \
                     'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], \
                         row[7], row[8], row[12], row[13], row[16]))


Total number of rows is:  8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("The number of rows in csv file is: ", sum(1 for line in f))

The number of rows in csv file is:  6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
conn = ["127.0.0.1"]

from cassandra.cluster import Cluster
try:
    cluster = Cluster(conn)

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project2 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
def execute_query(query):
    """ 
    This function will try to execute the query passed by function parameter, or
    print exception if execution of CQL query fails
    
    Parameter:
        query: CQL query to be executed
    """
    try:
        session.execute(query)
    except Exception as e:
        print(e)

In [9]:
def insert_statement(file, query, insert_dict):
    """
    This function reads csv file, extracts certain values from
    file by executing query, and casts certain data type of each
    column of data in line
    
    Parameters:
        file: file name ended with .csv  E.g. filename = "data123.csv"
        query: CQL query with INSERT statement
        insert_dict: Python dictionary format, 
                     Key: column index in csv file
                     Value: casted type
                     E.g. dict = {2: int} which refers to the third column of data
                                 to be casted by Integer
                                
    """
    
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f) # create reader object
        next(csvreader) # skip header
        for line in csvreader:
            session.execute(query, [y(line[x]) for x, y in insert_dict.items()])
        

In [10]:
def query_to_df(query):
    """
    This function will execute the SELECT query
    and return the result as DataFrame format
    
    Parameter:
        query: CQL query with SELECT statement
        
    """

    def pandas_factory(colnames, rows):
        """
        Inner function to return the pandas DataFrame with 
        paramter rows and columns corresponding to the function 
        parameters. The function asks Cassandra to transform
        all the rows in data into DataFrame format and column 
        names equal to the original colnames
        """
        return pd.DataFrame(rows, columns=colnames)
    
    # Assign the inner function to row_factory, return all the rows
    # into the DataFrame format
    session.row_factory = pandas_factory
    # Execute the query
    result = session.execute(query, timeout=None)
    # return the dataframe of all the rows
    df = result._current_rows
    return df

In [11]:
# query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# essionId = 338, and itemInSession = 4   
query1 = """CREATE TABLE IF NOT EXISTS song_playlist_session (
            sessionId int,
            itemInSession int,
            artist text, 
            song text, 
            length float, 
            PRIMARY KEY (sessionId, itemInSession))"""

# execute the CREATE query
execute_query(query1)

In [12]:
file = 'event_datafile_new.csv'
# INSERT query
query2 = """INSERT INTO song_playlist_session (
            sessionId, itemInSession, artist, song, length)
            VALUES (%s, %s, %s, %s, %s) """
# dict, key: colIndex, value: data type to be casted
dict1 = {8: int, 3: int, 0: str, 9: str, 5: float}

# call insert_statement function to insert certain values from file
insert_statement(file, query2, dict1)

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
# Add in the SELECT statement to verify the data was entered into the table
query3 = """SELECT artist, song, length 
             FROM song_playlist_session 
             WHERE sessionId = 338 AND itemInSession = 4 """

# call the function to see the result as DataFrame format
df1 = query_to_df(query3)
df1

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) 
# and user (first and last name) for userid = 10, sessionid = 182
query4 = """CREATE TABLE IF NOT EXISTS user_listen_record (
                 userId int, 
                 sessionId int, 
                 itemInSession int,
                 artist text, 
                 song text, 
                 firstName text, 
                 lastName text, 
                 PRIMARY KEY ((userId, sessionId), itemInsession))"""

# execute the CREATE query
execute_query(query4)

In [15]:
file = 'event_datafile_new.csv'
# INSERT query
query5 = """INSERT INTO user_listen_record (
            userId, sessionId, itemInSession, artist, song, firstName, lastName) 
            VALUES (%s, %s, %s, %s, %s, %s, %s)"""
# dict, key: colIndex, value: data type to be casted
dict2 = {10: int, 8: int, 3: int, 0: str, 9: str, 1: str, 4: str}

# call insert_statement function to insert certain values from file
insert_statement(file, query5, dict2)


In [16]:
# Add in the SELECT statement to verify the data was entered into the table
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) 
query6 = """SELECT artist, song, firstName, lastName
            FROM user_listen_record 
            WHERE userId = 10 AND sessionId = 182
            ORDER BY itemInSession DESC """

# call the function to see the result as DataFrame format
df2 = query_to_df(query6)
df2


,artist,song,firstname,lastname
0,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz
1,Sebastien Tellier,Kilometer,Sylvie,Cruz
2,Three Drives,Greece 2000,Sylvie,Cruz
3,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz


In [17]:
# Query 3: Give me every user name (first and last) in my music app history 
# who listened to the song 'All Hands Against His Own'
query7 = """CREATE TABLE IF NOT EXISTS get_user_list (
                    song text, 
                    userId int,
                    firstName text, 
                    lastName text, 
                    PRIMARY KEY (song, userId))"""

# execute the CREATE query
execute_query(query7)


In [18]:
file = 'event_datafile_new.csv'

# INSERT query
query8 = """INSERT INTO get_user_list (
            song, userId, firstName, lastName)
            VALUES (%s, %s, %s, %s)"""
            
# dict, key: colIndex, value: data type to be casted
dict3 = {9: str, 10: int, 1: str, 4: str}

# call insert_statement function to insert certain values from file
insert_statement(file, query8, dict3)

In [19]:
# Give me every user name (first and last) in my music app history who listened to 
# the song 'All Hands Against His Own'

# Add in the SELECT statement to verify the data was entered into the table
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) 
query9 = """SELECT firstName, lastName
            FROM get_user_list 
            WHERE song = 'All Hands Against His Own' """

# call the function to see the result as DataFrame format
df3 = query_to_df(query9)
df3


,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [20]:
# Drop the table before closing out the sessions
drop_query = ["drop table song_playlist_session", "drop table user_listen_record", "drop table get_user_list"]

# execute the drop queries
for query in drop_query:
    execute_query(query)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()